# Time and Seasonality

In [1]:
# base libraries
import pandas as pd
import math
import os
import json

In [2]:
# set variable from config file
config_path = os.path.abspath('..')

with open(config_path + '/config.json', 'r') as f:
    config = json.load(f)

processing_path = config['DEFAULT']['processing_path']
shipping_rot_filename = config['DEFAULT']['shipping_rot_filename']
seasonality_filename = config['DEFAULT']['seasonality_filename']

In [3]:
# import data
dtype_dic = {'MMSI':int,'dt':'str', 'lat':'float', 'long':'float','SOG':'float', 'rot':'float', 
             'Type':'str', 'gross_tonnage':'float','vessel_name':'str', 'ETA':'str', 'POC_LOCODE':'str',
             'last_port_LOCODE':'str', 'next_port_LOCODE':'str', 'status':'str','voyage_id':'float','tripid':int,
            'in_hazmat':'str','out_hazmat':'str'}
parse_dates = ['dt', 'ETA']

shipping_data = pd.read_csv(processing_path + shipping_rot_filename,header = 0,delimiter = ',',dtype = dtype_dic, parse_dates=parse_dates)


## Minute of the hour
Cyclical sine and cosine fields are also generated  

In [5]:
dt_framework = shipping_data[['MMSI','dt']].copy()
# + 1 to make it 1-60 instead of 0-59
dt_framework['MOH'] = dt_framework['dt'].dt.minute + 1
dt_framework['MOH1'] = dt_framework.apply(lambda x: math.sin((x['MOH']/60)*2*math.pi),axis = 1)
dt_framework['MOH2'] = dt_framework.apply(lambda x: math.cos((x['MOH']/60)*2*math.pi),axis = 1)

## Hour of the day
Cyclical sine and cosine fields are also generated

In [ ]:
dt_framework['HOD'] = dt_framework['dt'].dt.hour + 1
dt_framework['HOD1'] = dt_framework.apply(lambda x: math.sin((x['HOD']/24)*2*math.pi),axis = 1)
dt_framework['HOD2'] = dt_framework.apply(lambda x: math.cos((x['HOD']/24)*2*math.pi),axis = 1)

## Day of the week
Cyclical sine and cosine fields are also generated
(Monday = 1... Sunday = 7)

In [ ]:
dt_framework['DOW'] = dt_framework['dt'].dt.weekday + 1
dt_framework['DOW1'] = dt_framework.apply(lambda x: math.sin((x['DOW']/7)*2*math.pi),axis = 1)
dt_framework['DOW2'] = dt_framework.apply(lambda x: math.cos((x['DOW']/7)*2*math.pi),axis = 1)

## Week of year
Cyclical sine and cosine fields are also generated

In [ ]:
dt_framework['WOY'] = dt_framework['dt'].dt.week
dt_framework['WOY1'] = dt_framework.apply(lambda x: math.sin((x['WOY']/52)*2*math.pi),axis = 1)
dt_framework['WOY2'] = dt_framework.apply(lambda x: math.cos((x['WOY']/52)*2*math.pi),axis = 1)

## Month of year
Cyclical sine and cosine fields are also generated
(January = 1)

In [ ]:
dt_framework['MOY'] = dt_framework['dt'].dt.month
dt_framework['MOY1'] = dt_framework.apply(lambda x: math.sin((x['MOY']/12)*2*math.pi),axis = 1)
dt_framework['MOY2'] = dt_framework.apply(lambda x: math.cos((x['MOY']/12)*2*math.pi),axis = 1)

## Export Data

In [ ]:
dt_framework.to_csv(processing_path + seasonality_filename,header=True,index=False,sep=',')

In [ ]:
dt_framework.describe()